# 使用Marvin进行元数据提取和增强

本笔记本介绍如何使用[`Marvin`](https://github.com/PrefectHQ/marvin)从文本中提取和增强元数据。Marvin使用LLM来识别和提取元数据。元数据可以是从附加和增强的问题和答案到业务对象识别和阐述的任何内容。本笔记本将演示如何从csv文档中提取并阐述有关运动补充信息。

注意：您需要在下方提供有效的OpenAI密钥才能运行本笔记本。


## 设置


In [ ]:
%pip install llama-index-llms-openai
%pip install llama-index-extractors-marvin

In [ ]:
# !pip install marvin# !pip安装marvin

In [ ]:
from llama_index.core import SimpleDirectoryReader
from llama_index.llms.openai import OpenAI
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.extractors.marvin import MarvinMetadataExtractor

In [ ]:
import os
import openai

os.environ["OPENAI_API_KEY"] = "sk-..."

In [ ]:
# 限制文档文本长度documents[0].text = documents[0].text[:10000]

In [ ]:
import marvin
from marvin import ai_model

from llama_index.core.bridge.pydantic import BaseModel, Field

marvin.settings.openai.api_key = os.environ["OPENAI_API_KEY"]


@ai_model
class SportsSupplement(BaseModel):
    name: str = Field(..., description="The name of the sports supplement")
    description: str = Field(
        ..., description="A description of the sports supplement"
    )
    pros_cons: str = Field(
        ..., description="The pros and cons of the sports supplement"
    )

In [ ]:
llm_model = "gpt-3.5-turbo"# 构建文本分割器，将文本分割成块进行处理# 这需要一段时间来处理，您可以通过使用更大的chunk_size来增加处理时间# 当然，文件大小也是一个因素node_parser = TokenTextSplitter(    separator=" ", chunk_size=512, chunk_overlap=128)# 创建元数据提取器metadata_extractor = MarvinMetadataExtractor(    marvin_model=SportsSupplement, llm_model_string=llm_model)  # 让我们为每个节点提取自定义实体。# 使用node_parser从文档中获取节点from llama_index.core.ingestion import IngestionPipelinepipeline = IngestionPipeline(transformations=[node_parser, metadata_extractor])nodes = pipeline.run(documents=documents, show_progress=True)

In [ ]:
from pprint import pprint

for i in range(5):
    pprint(nodes[i].metadata)

{'marvin_metadata': {'description': 'L-arginine alpha-ketoglutarate',
                     'name': 'AAKG',
                     'pros_cons': '1.0, peak power output, strength–power, '
                                  'weight training, OTW, 242, 1, 20, nan, A '
                                  '2006 study found AAKG supplementation '
                                  'improved maximum effort 1-repetition bench '
                                  'press and Wingate peak power performance.'}}
{'marvin_metadata': {'description': 'Gulping down baking soda (sodium '
                                    'bicarbonate) makes the blood more '
                                    'alkaline, improving performance in '
                                    'lactic-acid-fueled events like the 800m '
                                    'sprint.',
                     'name': 'Baking soda',
                     'pros_cons': 'Downside: a badly upset stomach.'}}
{'marvin_metadata': {'description': 'Branch